# Final Project





## Links

# Main libraries

In [17]:
import nbimporter
import json
import os

# Information TSpec-LLM Dataset

In [13]:
import Process_tspec_llm

processTspec = Process_tspec_llm

## Verify TSpec-LLM Data

In [14]:
directory_path = '../Dataset/TSpec-LLM/3GPP-clean'
md_sizes_report = processTspec.calculate_md_sizes_and_word_count(directory_path)
json_filename = 'md_sizes_word_count_report.json'
processTspec.save_report_to_json(md_sizes_report, json_filename)
processTspec.print_summary(md_sizes_report)


Total size of .md files in the repository: 1330.64 MB
Total words in .md files in the repository: 164572408
Rel-17: Total size of .md files: 736.61 MB, Total words: 90746969
Rel-18: Total size of .md files: 594.03 MB, Total words: 73825439


## Get texts in TSpec-LLM

In [15]:
# Exemplo de uso
directory_path = '../Dataset/TSpec-LLM/3GPP-clean'
tspec_data = processTspec.load_tspec_data(directory_path)

# Verificar um exemplo
print(f"Total de documentos carregados: {len(tspec_data)}")
print(f"Exemplo: {tspec_data[0]}")

Total de documentos carregados: 2788
Exemplo: {'release': 'Rel-17', 'series': '48_series', 'content': '![](media/image1.wmf){width="7.0875in" height="0.9722222222222222in"}\n\nContents {#contents .TT}\n========\n\n[7](#foreword)\n\n[8](#scope)\n\n[8](#references)\n\n[9](#abbreviations)\n\n[9](#protocol-model)\n\n[10](#radio-link-layer-management-procedures)\n\n[10](#link-establishment-indication)\n\n[11](#link-establishment-request)\n\n[11](#link-release-indication)\n\n[11](#link-release-request)\n\n[12](#transmission-of-a-transparent-l3-message-in-acknowledged-mode)\n\n[12](#reception-of-a-transparent-l3-message-in-acknowledged-mode)\n\n[12](#transmission-of-a-transparent-l3-message-in-unacknowledged-mode)\n\n[12](#reception-of-a-transparent-l3-message-in-unacknowledged-mode)\n\n[13](#link-error-indication)\n\n[13](#dedicated-channel-management-procedures)\n\n[13](#channel-activation)\n\n[13](#signalling-procedure)\n\n[14](#activation-for-intra-cell-channel-change)\n\n[15](#activation

# Question Dataset

In [18]:
# Path to the TeleQnA processed question in JSON file
rel17_100_questions_path = r"../Files/rel17_100_questions.json"

# Load the TeleQnA data just release 17
with open(rel17_100_questions_path, "r", encoding="utf-8") as file:
    rel17_100_questions = json.load(file)
print(len(rel17_100_questions))

100


In [19]:
rel17_100_questions[0]

{'question': 'Which NGAP procedure is used for inter-system load balancing? [3GPP Release 17]',
 'option 1': 'eNB Configuration Transfer',
 'option 2': 'Downlink RAN Configuration Transfer',
 'option 3': 'Uplink RAN Configuration Transfer',
 'option 4': 'MME Configuration Transfer',
 'answer': 'option 3: Uplink RAN Configuration Transfer',
 'explanation': 'The NGAP procedure used for inter-system load balancing is Uplink RAN Configuration Transfer.',
 'category': 'Standards overview'}

# OpenAi Model

In [20]:
import openai
import os

# Open AI Key
api_key=os.getenv("OPENAI_API_KEY")

In [ ]:
# # Obter a primeira pergunta
# first_question = rel17_100_questions[0]["question"]

# from openai import OpenAI
# client = OpenAI(api_key=api_key)

# completion = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {
#             "role": "user",
#             "content": first_question
#         }
#     ]
# )

# # Extrair a resposta do modelo
# answer = completion.choices[0].message

# # Exibir a resposta
# print(f"Pergunta: {first_question}")
# print(f"Resposta do GPT-3.5-turbo: {answer}")


# Llama from Ollama Model

In [21]:
from langchain.llms import Ollama

# Initialize the Ollama model with LLaMA 3.1 8B
llama_model = Ollama(model="llama3.1")

In [36]:
# Função para matar o processo ollama_llama_server
def kill_ollama_server():
    os.system("pkill -f ollama_llama_server")  # Comando para matar o processo

# Função para processar e rodar o modelo com uma única pergunta
def run_llama(question):

    # Definir o prompt para o modelo
    prompt = f"""
    {question}
    """
    response = llama_model.invoke(prompt)
    # Finalizar o processo ollama_llama_server
    kill_ollama_server()

    return response

In [37]:
question = rel17_100_questions[0]['question']
print(question)

Which NGAP procedure is used for inter-system load balancing? [3GPP Release 17]


In [38]:
run_llama(question)

'A very specific question!\n\nAccording to the 3GPP specification (Release 17), the NGAP (5G Network Application Protocol) procedure used for inter-system load balancing is:\n\n**NGAP_PING**\n\nThe NGAP PING procedure is a mechanism that allows the NG-RAN (5G Radio Access Network) to send periodic PING messages to the AMF (Access and Mobility Management Function). The AMF can then use this information to monitor the traffic load in different 5G RANs and adjust the traffic distribution accordingly.\n\nSo, the answer is: **NGAP_PING**.'

# Build Dataset and Save in File

## Pre definitions

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install langchain sentence-transformers faiss-cpu torch

In [3]:
# !pip install tf-keras

In [4]:
# !pip install langchain langchain_community langchain-ollama

In [1]:
from sentence_transformers import SentenceTransformer
import torch
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pickle
import faiss
import numpy as np
import random

/home/arimatea/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-10-02 09:53:44.142508: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-02 09:53:44.283689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 09:53:44.339544: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already bee

In [2]:
import torch
print("CUDA Available: ", torch.cuda.is_available())
print("CUDA Device Name: ", torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

# Verificar se CUDA está disponível para acelerar o processamento
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

CUDA Available:  True
CUDA Device Name:  NVIDIA GeForce RTX 3050 Ti Laptop GPU
Usando dispositivo: cuda


## Dataset Create and Save

In [4]:
# Caminho para o seu arquivo JSON
caminho_arquivo = 'context_articles.json'

# Carregar os dados do JSON
with open(caminho_arquivo, 'r', encoding='utf-8') as file:
    dataset = json.load(file)
dataset = list(dataset.items())

In [5]:
type(dataset)

list

In [6]:
print(f"Total: {len(dataset)}")
print("Ex:", dataset[0])
# print("Ex: ", next(iter(dataset.items())))

Total: 56550
Ex: ('san diego padres', 'The San Diego Padres are an American <a href="professional%20baseball">professional baseball</a> team based in <a href="San%20Diego">San Diego</a>, <a href="California">California</a>. They compete in <a href="Major%20League%20Baseball">Major League Baseball</a> (MLB) as a member club of the <a href="National%20League">National League</a> (NL) <a href="National%20League%20West">West division</a>. Founded in <a href="1969%20San%20Diego%20Padres%20season">1969</a>, the Padres have won two <a href="List%20of%20National%20League%20pennant%20winners">NL pennants</a> — in <a href="1984%20San%20Diego%20Padres%20season">1984</a> and <a href="1998%20San%20Diego%20Padres%20season">1998</a>, losing in the <a href="World%20Series">World Series</a> both years. As of <a href="2017%20San%20Diego%20Padres%20season">2018</a>, they have had 14 winning seasons in franchise history. The Padres are one of two Major League Baseball teams (the other being the <a href="L

In [7]:
# Inicializar o Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,      # Tamanho máximo de cada chunk
    chunk_overlap=100,    # Quantidade de sobreposição entre os chunks
    separators=["\n\n", "\n", " ", ""]  # Separadores para segmentação
)

# Função para dividir o texto em chunks
def dividir_em_chunks(texto):
    return text_splitter.split_text(texto)

# Aplicar a divisão em todos os textos do dataset
dataset_chunks = []
for book in dataset:
    label = book[0]      # O primeiro elemento da tupla é o rótulo
    texto = book[1]      # O segundo elemento da tupla é o texto
    chunks = dividir_em_chunks(texto)
    for chunk in chunks:
        dataset_chunks.append({
            'text': chunk,
            'label': label  # Preservar o rótulo
        })


In [8]:
print(f"Total chunks: {len(dataset_chunks)}")
print("Exemple chunk:", dataset_chunks[0])

Total chunks: 769805
Exemple chunk: {'text': 'The San Diego Padres are an American <a href="professional%20baseball">professional baseball</a> team based in <a href="San%20Diego">San Diego</a>, <a href="California">California</a>. They compete in <a href="Major%20League%20Baseball">Major League Baseball</a> (MLB) as a member club of the <a href="National%20League">National League</a> (NL) <a href="National%20League%20West">West division</a>. Founded in <a href="1969%20San%20Diego%20Padres%20season">1969</a>, the Padres have won two <a href="List%20of%20National%20League%20pennant%20winners">NL pennants</a> — in <a href="1984%20San%20Diego%20Padres%20season">1984</a> and <a href="1998%20San%20Diego%20Padres%20season">1998</a>, losing in the <a href="World%20Series">World Series</a> both years. As of <a href="2017%20San%20Diego%20Padres%20season">2018</a>, they have had 14 winning seasons in franchise history. The Padres are one of two Major League Baseball teams (the other being the <a 

In [9]:
# # Salvar embeddings em um arquivo .pkl (pickle)
# with open('dataset_chunks.pkl', 'wb') as f:
#     pickle.dump(dataset_chunks, f)

In [10]:
# Carregar embeddings do arquivo .pkl
with open('dataset_chunks.pkl', 'rb') as f:
    dataset_chunks = pickle.load(f)

In [11]:
num_test_samples = 100
data_test = dataset_chunks[:num_test_samples]

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

# Carregar o modelo
modelo = SentenceTransformer('all-mpnet-base-v2', device=device)

# Gerar embeddings para cada chunk
def gerar_embeddings(documents, batch_size=64):
    textos = [doc['text'] for doc in documents]
    embeddings = modelo.encode(
        textos, 
        convert_to_tensor=False, 
        show_progress_bar=True, 
        batch_size=batch_size
    )
    return embeddings

# Obter os embeddings
# embeddings = gerar_embeddings(dataset_chunks)
embeddings_test = gerar_embeddings(data_test)


In [ ]:
# Verificar o tamanho dos embeddings
print(f"Size of embeddings: {len(embeddings)}")
print(f"Sixe of each embedding: {len(embeddings[1])}")

Size of embeddings: 769805
Sixe of each embedding: 768


In [ ]:
# # Salvar embeddings em um arquivo .pkl (pickle)
# with open('embeddings.pkl', 'wb') as f:
#     pickle.dump(embeddings, f)

In [ ]:
# Carregar embeddings do arquivo .pkl
with open('embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

print(f"Loaded Dataset size: {len(embeddings)}")


Loaded Dataset size: 769805


In [ ]:
embeddings[1]

array([-4.39035557e-02,  4.13923413e-02, -1.31815979e-02, -9.05656070e-03,
        1.88980866e-02, -2.30787601e-03,  1.14673702e-02,  2.85354629e-02,
        5.36813587e-02, -2.80141756e-02, -2.59313527e-02,  3.96644957e-02,
        2.14470923e-02, -3.91444676e-02, -3.48327197e-02,  3.48632038e-02,
       -1.66153293e-02,  1.04179047e-02,  5.61972242e-03, -8.13380815e-03,
       -6.25603646e-02,  2.35460140e-02, -3.62865552e-02, -1.31997420e-02,
       -6.61428366e-03, -2.34690234e-02, -7.83822779e-03, -3.77242044e-02,
       -1.17200012e-04, -6.24527708e-02,  1.96148511e-02, -3.35057899e-02,
        1.22996578e-02, -4.68037836e-03,  2.37095355e-06, -1.14677325e-02,
        1.60544105e-02, -2.29683928e-02, -1.00952117e-02, -2.44158488e-02,
       -2.11209077e-02, -1.79950334e-02,  4.00784612e-02,  2.60065775e-02,
        1.61408447e-03, -5.84057346e-02,  4.41867001e-02, -8.33072290e-02,
       -2.39038770e-03,  1.27016045e-02,  3.04946639e-02, -2.57647745e-02,
       -7.96181187e-02, -

# Test (Questions) Dataset

In [17]:
# Carregar resultados de um arquivo JSON
with open('iirc_test.json', 'r') as f:
    val_dataset = json.load(f)

In [8]:
# Exibir as primeiras linhas do dataset para entender sua estrutura
print(val_dataset[0])

{'questions': [{'answer': {'type': 'span', 'answer_spans': [{'text': 'sky and thunder god', 'passage': 'zeus', 'type': 'answer', 'start': 83, 'end': 102}]}, 'question': 'What is Zeus know for in Greek mythology?', 'context': [{'text': 'he Palici the sons of Zeus', 'passage': 'main', 'indices': [684, 710]}, {'text': 'in Greek mythology', 'passage': 'main', 'indices': [137, 155]}, {'text': 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion', 'passage': 'Zeus', 'indices': [0, 110]}], 'question_links': ['Greek mythology', 'Zeus']}], 'text': "The Palici (Παλικοί in Greek), or Palaci, were a pair of indigenous Sicilian chthonic deities in Roman mythology, and to a lesser extent in Greek mythology. They are mentioned in Ovid's Metamorphoses V, 406, and in Virgil's Aeneid IX, 585. Their cult centered on three small lakes that emitted sulphurous vapors in the Palagonia plain, and as a result these twin brothers were associated with ge

In [9]:

# Conjunto para armazenar tipos de resposta únicos
answer_types = set()

# Iterar sobre o dataset para coletar todos os tipos de resposta
for entry in val_dataset:
    for question in entry['questions']:
        answer = question.get('answer')
        if answer:
            answer_type = answer.get('type')
            if answer_type:
                answer_types.add(answer_type)

# Exibir os tipos únicos de resposta encontrados
print("Tipos de resposta encontrados:")
for ans_type in answer_types:
    print(ans_type)

Tipos de resposta encontrados:
span
value
none
binary


In [18]:
# Função para verificar se a pergunta tem uma resposta válida
def has_valid_answer(question):
    # Verifica se 'answer' está presente e não é 'none'
    answer = question.get('answer')
    # if answer and answer.get('type') != 'none' and answer.get('type') != 'value' and answer.get('type') != 'span':
    if answer and answer.get('type') != 'none':
        return True
    return False

# Lista para armazenar perguntas e respostas válidas
questions_and_answers = []

# Iterar sobre o dataset
for entry in val_dataset:
    for question in entry['questions']:
        if has_valid_answer(question):
            question_text = question['question']
            answer_data = question['answer']
            # Armazenar a pergunta e a resposta
            questions_and_answers.append({
                'question': question_text,
                'answer': answer_data
            })

# Exibir algumas das perguntas e respostas válidas
for i, q_a in enumerate(questions_and_answers[:5]):
    print(f"Pergunta {i+1}: {q_a['question']}")
    print(f"Resposta: {q_a['answer']}\n")

# Total de perguntas com respostas válidas
print(f"Total de perguntas com respostas válidas: {len(questions_and_answers)}")

random.shuffle(questions_and_answers)

questions = questions_and_answers[:150]

print(f"Total de perguntas com respostas válidas: {len(questions)}")

Pergunta 1: What is Zeus know for in Greek mythology?
Resposta: {'type': 'span', 'answer_spans': [{'text': 'sky and thunder god', 'passage': 'zeus', 'type': 'answer', 'start': 83, 'end': 102}]}

Pergunta 2: How long had the First World War been over when Messe was named aide-de-camp?
Resposta: {'answer_value': '5', 'type': 'value', 'answer_unit': 'years'}

Pergunta 3: How old was Messe when the First World War started?
Resposta: {'answer_value': '30', 'type': 'value', 'answer_unit': 'years'}

Pergunta 4: How long had Angela Scoular been acting professionally when she appeared in the movie "On Her Majesty's Secret Service"?
Resposta: {'answer_value': '2', 'type': 'value', 'answer_unit': 'years'}

Pergunta 5: What is the capacity of the stadium where Brunt returned to action after a torn ACL?
Resposta: {'type': 'span', 'answer_spans': [{'text': '26,688', 'passage': 'the hawthorns', 'type': 'answer', 'start': 136, 'end': 142}]}

Total de perguntas com respostas válidas: 887
Total de pergu

In [3]:
def get_answer(answer):
    """
    Retorna a resposta correta dependendo do tipo.
    
    Para respostas do tipo 'span', retorna o campo 'text'.
    Para respostas do tipo 'value', retorna o campo 'answer_value'.
    """
    answer_type = answer.get('type')
    # print(f"answer_type: {answer_type}")

    if answer_type == 'span':
        # Para respostas do tipo 'span', pegar o campo 'text' de 'answer_spans'
        if 'answer_spans' in answer:
            return answer['answer_spans'][0].get('text')
        else:
            return None  # Caso não haja 'answer_spans', retorna None
    
    elif answer_type == 'value' or answer_type == 'binary':
        # Para respostas do tipo 'value', pegar o 'answer_value'
        return answer.get('answer_value')
    
    return None  # Caso o tipo de resposta não seja reconhecido

In [12]:
from collections import defaultdict

# Contar quantas perguntas com respostas válidas de cada tipo existem
answer_type_counts = defaultdict(int)

for qa in questions:
    answer_type = qa['answer'].get('type')
    if answer_type:
        answer_type_counts[answer_type] += 1

# Exibir a contagem de perguntas por tipo de resposta
print("Contagem de perguntas por tipo de resposta:")
for answer_type, count in answer_type_counts.items():
    print(f"{answer_type}: {count}")

# Total de perguntas selecionadas
print(f"Total de perguntas com respostas válidas: {len(questions)}")

Contagem de perguntas por tipo de resposta:
span: 88
value: 51
binary: 11
Total de perguntas com respostas válidas: 150


In [13]:
test = questions[0]['answer']
print(test)
print(get_answer(test))

{'type': 'span', 'answer_spans': [{'text': 'Alfred Nobel', 'passage': 'dynamite', 'type': 'answer', 'start': 182, 'end': 194}]}
Alfred Nobel


In [62]:
# # Salvar embeddings em um arquivo .pkl (pickle)
# with open('final_questions.pkl', 'wb') as f:
#     pickle.dump(questions, f)

# Load Dataset from Files and Build Query

In [4]:
# Carregar os metadados com embeddings
with open('embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)
    
# Carregar os metadados com dataset_chunks
with open('dataset_chunks.pkl', 'rb') as f:
    dataset_chunks = pickle.load(f)


In [5]:
# Carregar embeddings do arquivo .pkl
with open('final_questions.pkl', 'rb') as f:
    questions = pickle.load(f)

print(f"Loaded Dataset size: {len(questions)}")

Loaded Dataset size: 150


In [15]:
embeddings[0]

array([-6.71826154e-02,  4.04281840e-02, -1.50079848e-02, -1.27104772e-02,
       -6.16077520e-03, -2.40212050e-03, -5.90160154e-02, -2.58078193e-03,
        3.17934304e-02, -3.35996486e-02, -4.48811427e-02,  4.57187258e-02,
        1.88907702e-02, -3.55389565e-02, -6.80492371e-02, -5.63734258e-03,
       -1.72109548e-02,  1.82366017e-02, -2.71648150e-02, -1.41241457e-02,
       -1.38981622e-02, -2.02171113e-02, -3.63142341e-02, -1.94650572e-02,
       -1.58506837e-02, -2.45793746e-03,  1.77203550e-03, -4.34064902e-02,
       -1.35222757e-02, -8.24181959e-02,  4.96267155e-02, -1.59301627e-02,
       -8.56843591e-03, -2.90886257e-02,  2.27898545e-06, -3.57254334e-02,
        3.54990512e-02, -3.04277800e-02,  2.68279649e-02, -1.76609512e-02,
        8.22210871e-03,  2.70059761e-02,  3.38868387e-02,  9.31891892e-03,
        1.46788266e-02, -8.70674551e-02,  6.93292618e-02,  6.83908612e-02,
        2.89525203e-02, -3.59135866e-02,  3.17896456e-02, -3.78060713e-02,
       -2.63794065e-02,  

In [16]:
dataset_chunks[0]

{'text': 'The San Diego Padres are an American <a href="professional%20baseball">professional baseball</a> team based in <a href="San%20Diego">San Diego</a>, <a href="California">California</a>. They compete in <a href="Major%20League%20Baseball">Major League Baseball</a> (MLB) as a member club of the <a href="National%20League">National League</a> (NL) <a href="National%20League%20West">West division</a>. Founded in <a href="1969%20San%20Diego%20Padres%20season">1969</a>, the Padres have won two <a href="List%20of%20National%20League%20pennant%20winners">NL pennants</a> — in <a href="1984%20San%20Diego%20Padres%20season">1984</a> and <a href="1998%20San%20Diego%20Padres%20season">1998</a>, losing in the <a href="World%20Series">World Series</a> both years. As of <a href="2017%20San%20Diego%20Padres%20season">2018</a>, they have had 14 winning seasons in franchise history. The Padres are one of two Major League Baseball teams (the other being the <a href="Los%20Angeles%20Angels">Los An

In [6]:
# Extrair os embeddings como um array NumPy
embeddings_np = np.array([embedding for embedding in embeddings]).astype('float32')
# embeddings_np = np.array([embedding for embedding in embeddings])

# Normalizar os embeddings para usar similaridade de cosseno
faiss.normalize_L2(embeddings_np)

# Dimensionalidade dos embeddings
dim = embeddings_np.shape[1]
print(f"Dimensionalidade dos embeddings: {dim}")

# Escolher o tipo de índice FAISS
# Usaremos IndexFlatIP para similaridade de cosseno após normalização
# Calcula o produto interno entre o embedding salvo e o de consulta
index = faiss.IndexFlatIP(dim)

# Verificar se o índice foi criado corretamente
print("Índice FAISS criado:", index.is_trained)

# Adicionar os embeddings ao índice
index.add(embeddings_np)
print(f"Total de embeddings no índice: {index.ntotal}")

# Salvar o índice FAISS para uso futuro
faiss.write_index(index, 'faiss_index.bin')
print("Índice FAISS salvo com sucesso em 'faiss_index.bin'.")


Dimensionalidade dos embeddings: 768
Índice FAISS criado: True
Total de embeddings no índice: 769805
Índice FAISS salvo com sucesso em 'faiss_index.bin'.


In [7]:
del index
del embeddings

In [8]:
def query_indice(query, k=5, caminho_indice='faiss_index.bin', modelo=SentenceTransformer('all-mpnet-base-v2', device=device)):
    """
    Realiza uma consulta no índice FAISS e retorna os top k resultados.
    
    Args:
        query (str): Texto da query.
        k (int): Número de resultados a retornar.
        caminho_indice (str): Caminho para o arquivo FAISS index.
        caminho_metadados (str): Caminho para o arquivo de metadados.
        
    Returns:
        List of tuples: Cada tupla contém a similaridade e o metadado correspondente.
    """
    # Carregar o índice FAISS
    index = faiss.read_index(caminho_indice)
    
    # Gerar embedding para a query
    embedding_consulta = modelo.encode([query], convert_to_tensor=False, show_progress_bar=False)
    embedding_consulta_np = np.array(embedding_consulta).astype('float32')
    # embedding_consulta_np = np.array(embedding_consulta)
    faiss.normalize_L2(embedding_consulta_np)
    
    # Realizar a busca no índice
    D, I = index.search(embedding_consulta_np, k)
    # print(f"{D} e {I}")
    
    resultados = []
    for dist, idx in zip(D[0], I[0]):
        if idx < len(dataset_chunks):
            metadado = dataset_chunks[idx]
            resultados.append((dist, metadado))
    return resultados

/home/arimatea/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# Teste de query
query = "san diego padres"
# query = "located in San Diego"
resultados = query_indice(query, k=3)

print("Resultados da query:")
for dist, metadado in resultados:
    print(f"Similaridade: {dist:.4f}")
    print(f"Título: {metadado['label']}")
    print(f"Texto: {metadado['text'][:200]}...")  # Exibir os primeiros 200 caracteres
    print("-" * 50)

Resultados da query:
Similaridade: 0.6766
Título: san diego padres
Texto: The San Diego Padres are an American <a href="professional%20baseball">professional baseball</a> team based in <a href="San%20Diego">San Diego</a>, <a href="California">California</a>. They compete in...
--------------------------------------------------
Similaridade: 0.6616
Título: world series
Texto: Diego Padres</a> for bragging rights in Southern California, although all three of those teams have appeared in the World Series at various times....
--------------------------------------------------
Similaridade: 0.6415
Título: national league west
Texto: Current members.<ul><li>- <a href="Arizona%20Diamondbacks">Arizona Diamondbacks</a> - Joined in 1998 as an expansion team
</li><li>- <a href="Colorado%20Rockies">Colorado Rockies</a> - Joined in 1993 ...
--------------------------------------------------


# Ollama

In [9]:
def get_information(resultados):
    """
    Formata as informações dos resultados exibindo apenas o texto, sem o título.

    Args:
        resultados (list): Lista de tuplas contendo a similaridade e o metadado correspondente.

    Returns:
        str: Um texto formatado contendo as informações de cada resultado.
    """
    information_text = ""
    
    # Iterar sobre os resultados e formatar as informações
    for idx, (_, metadado) in enumerate(resultados, start=1):
        # Pega o texto associado
        # texto = metadado.get('text', 'Sem texto')
        texto = metadado['text']
        
        # Formata a saída para cada informação
        information_text += f"\nInformation {idx}:\n{metadado['label']}\n{texto}...\n"  # Mostra os primeiros 200 caracteres
        # information_text += f"Information {idx}:\n{texto[:200]}...\n"  # Mostra os primeiros 200 caracteres
    
    return information_text

In [10]:
# Teste da função
query = "san diego padres"
resultados = query_indice(query, k=3)

# Gerar as informações formatadas
informations = get_information(resultados)

# Exibir o resultado
print(informations)


Information 1:
san diego padres
The San Diego Padres are an American <a href="professional%20baseball">professional baseball</a> team based in <a href="San%20Diego">San Diego</a>, <a href="California">California</a>. They compete in <a href="Major%20League%20Baseball">Major League Baseball</a> (MLB) as a member club of the <a href="National%20League">National League</a> (NL) <a href="National%20League%20West">West division</a>. Founded in <a href="1969%20San%20Diego%20Padres%20season">1969</a>, the Padres have won two <a href="List%20of%20National%20League%20pennant%20winners">NL pennants</a> — in <a href="1984%20San%20Diego%20Padres%20season">1984</a> and <a href="1998%20San%20Diego%20Padres%20season">1998</a>, losing in the <a href="World%20Series">World Series</a> both years. As of <a href="2017%20San%20Diego%20Padres%20season">2018</a>, they have had 14 winning seasons in franchise history. The Padres are one of two Major League Baseball teams (the other being the <a href="Los%20A

In [10]:
from langchain.llms import Ollama

# Initialize the Ollama model with LLaMA 3.1 8B
llama_model = Ollama(model="llama3.1")

In [11]:
import os

# Função para matar o processo ollama_llama_server
def kill_ollama_server():
    os.system("pkill -f ollama_llama_server")  # Comando para matar o processo

# Função para processar e rodar o modelo com uma única pergunta
def run_llama(question):
    # Extrair a pergunta e a resposta correta
    text = question['question']
    # answer = question['answer']
    # answer = get_answer(answer)

    # Processar consulta e obter informações
    query = query_indice(text, k=3)
    informations = get_information(query)

    # Definir o prompt para o modelo
    prompt = f"Think in this informations:\n{informations}\nGiven the Informations try to answer the Question\nQuestion: {text}\nIn the end of your text just give your final answer in format:\nAnswer: <answer>"
    del query
    del informations
    # print(prompt)

    # Rodar o modelo e obter a resposta
    response = llama_model.invoke(prompt)

    # Finalizar o processo ollama_llama_server
    kill_ollama_server()

    return response

In [32]:
# Exemplo de pergunta seguindo o mesmo formato
question_example = {
    'question': "How long had the First World War been over when Messe was named aide-de-camp?",
    'answer': {'answer_value': '5', 'type': 'value', 'answer_unit': 'years'}
}

question_example = questions[0]
# Executar a função com a pergunta de exemplo
response = run_llama(question_example)

Think in this informations:

Information 1:
dynamite
In 1857, Nobel filed the first of several hundred patents, mostly concerning air pressure, gas and fluid gauges, but remained fascinated with nitroglycerin's potential as an explosive. Nobel, along with his father and brother <a href="Emil%20Oskar%20Nobel">Emil</a>, experimented with various combinations of nitroglycerin and black powder. Nobel came up with a solution of how to safely detonate nitroglycerin by inventing the <a href="detonator">detonator</a>, or blasting cap, that allowed a controlled explosion set off from a distance using a <a href="Fuse%20%28explosives%29">fuse</a>. In the summer of 1863, Nobel performed his first successful detonation of pure nitroglycerin, using a blasting cap made of a copper <a href="percussion%20cap">percussion cap</a> and <a href="Mercury%28II%29%20fulminate">mercury fulminate</a>. In 1864, Alfred Nobel filed patents for both the blasting cap and his method of synthesizing nitroglycerin, usin

In [35]:
# Imprimir o resultado
question_example
print("Response from LLaMA 3.1:", response)
print("Real response:", get_answer(question_example['answer']))

Response from LLaMA 3.1: After analyzing the information, I can conclude that:

Dynamite was invented by Alfred Nobel, and is mentioned as the type of explosive used by Bohlwinkel's henchman. However, there is no direct connection between Nobel and the event described in Information 2.

However, Information 1 mentions that in 1864, Alfred Nobel filed patents for both the blasting cap and his method of synthesizing nitroglycerin. This implies that Nobel was working on explosive technologies around the time period when Bohlwinkel's henchman might have used dynamite.

But, there is no direct link between Nobel and the specific event in which Bohlwinkel's henchman plants dynamite on the Aurora. Given the lack of information about who specifically invented or used dynamite in this context, I can only speculate that it was likely based on some variant of nitroglycerin-based explosives, but since Alfred Nobel is credited with working on explosive technologies around this time period and inven

# Calculate F1-Score

In [48]:
def preprocess_answer(llama_response):
    # Divide a resposta em linhas
    lines = llama_response.strip().split('\n')
    
    # Verifica se a última linha contém "Answer:"
    if lines and "Answer:" in lines[-1]:
        # Extrai a parte após "Answer:"
        answer_line = lines[-1].strip()  # Remove espaços em branco da linha
        answer = answer_line.split("Answer:")[-1].strip()  # Pega a parte após "Answer:"
        return answer
    # Majoritariamente a resposta está na última linha, porém usar a resposta toda do modelo não alterou tanto o resultado final
    return lines[-1]
    # return llama_response

In [14]:
import nltk

# Baixar o pacote de stopwords
nltk.download('stopwords')

# Verificar se as stopwords foram baixadas corretamente
from nltk.corpus import stopwords

# Carregar stopwords em inglês
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/arimatea/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
import collections
import re
# from sklearn.metrics import f1_score
from nltk.corpus import stopwords

# Carregar lista de stopwords em inglês
stop_words = set(stopwords.words('english'))

# Função para tokenizar o texto e remover stopwords
def get_tokens(text):
    tokens = re.findall(r'\w+', text.lower())
    # Remover stopwords
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

def compute_f1(real, pred):
    real_tokens = get_tokens(real)
    # print("Real tokens:", real_tokens)
    pred = preprocess_answer(pred)
    pred_tokens = get_tokens(pred)
    # print("Predicted tokens:", pred_tokens)
    
    common = collections.Counter(real_tokens) & collections.Counter(pred_tokens)
    num_same = sum(common.values())
    
    if len(real_tokens) == 0 or len(pred_tokens) == 0:
        # Se qualquer resposta estiver vazia, F1 é 1 se estiverem de acordo, 0 caso contrário
        return int(real_tokens == pred_tokens)
    
    if num_same == 0:
        return 0
    
    precision = 1.0 * num_same / len(pred_tokens)
    recall = 1.0 * num_same / len(real_tokens)
    
    # Cálculo do F1-Score
    if precision + recall == 0:
        return 0
    f1 = (2 * recall * precision) / (precision + recall)
    return f1

In [48]:
answer_test = get_answer(question_example['answer'])
response_test = response

In [46]:
preprocess_answer(answer_test)

'Alfred Nobel'

In [51]:
f1_score = compute_f1(answer_test, response_test)
print(f"F1 Score: {f1_score:.4f}")

Real tokens: ['alfred', 'nobel']
Predicted tokens: ['alfred', 'nobel']
F1 Score: 1.0000


# Mean F1-Score in Val Dataset

In [14]:
# Função para processar todo o dataset e calcular o F1 score
def evaluate_questions(questions):
    total_f1 = 0
    f1_count_exact = 0
    num_questions = len(questions)
    responses = []
    i = 0

    for question in questions:
        response = run_llama(question)
        responses.append(response)
        
        answer = question['answer']
        answer = get_answer(answer)
        
        # Calcular o F1 score
        f1 = compute_f1(answer, response)  # Passar somente a resposta
        total_f1 += f1
        
        if f1 == 1:
            f1_count_exact += 1
        i += 1
        print(f"Número de respostas percorridas: {i}")
    
    average_f1 = total_f1 / num_questions if num_questions > 0 else 0
    return average_f1, f1_count_exact, responses

In [15]:
questions[0]

{'question': "Who invented the explosive that Bohlwinkel's henchman plants on the Aurora?",
 'answer': {'type': 'span',
  'answer_spans': [{'text': 'Alfred Nobel',
    'passage': 'dynamite',
    'type': 'answer',
    'start': 182,
    'end': 194}]}}

In [27]:
# Testa a iteração somente sobre 2 perguntas

questions_test = questions[:2]

average_f1, exact_count, responses = evaluate_questions(questions_test)
print("Média do F1 Score:", average_f1)
print("Número de vezes que o F1 Score foi igual a 1:", exact_count)

Número de respostas percorridas: 1
Número de respostas percorridas: 2
Média do F1 Score: 0.5
Número de vezes que o F1 Score foi igual a 1: 1


In [28]:
responses[0]

'To determine who invented the explosive that Bohlwinkel\'s henchman plants on the Aurora, we can refer to Information 1 and Information 2.\n\nAccording to Information 1, Alfred Nobel filed patents for both the blasting cap and his method of synthesizing nitroglycerin in 1864. The text also mentions that he had previously experimented with various combinations of nitroglycerin and black powder to safely detonate nitroglycerin.\n\nHowever, upon closer inspection, we can see that Information 1 talks about Alfred Nobel inventing the blasting cap for nitroglycerin, but it does not specifically mention dynamite. Dynamite is actually mentioned in Information 2 as part of Bohlwinkel\'s sabotage attempts against Tintin and Captain Haddock.\n\nUpon further investigation, Information 3 does not provide any information relevant to this question.\n\nGiven that dynamite is the explosive used by Bohlwinkel\'s henchman, we can see that the only relevant piece of information comes from Information 2, 

In [17]:
# Itera sobre todas as 150 perguntas

average_f1, exact_count, responses = evaluate_questions(questions)
print("Média do F1 Score:", average_f1)
print("Número de vezes que o F1 Score foi igual a 1:", exact_count)

Número de respostas percorridas: 1
Número de respostas percorridas: 2
Número de respostas percorridas: 3
Número de respostas percorridas: 4
Número de respostas percorridas: 5
Número de respostas percorridas: 6
Número de respostas percorridas: 7
Número de respostas percorridas: 8
Número de respostas percorridas: 9
Número de respostas percorridas: 10
Número de respostas percorridas: 11
Número de respostas percorridas: 12
Número de respostas percorridas: 13
Número de respostas percorridas: 14
Número de respostas percorridas: 15
Número de respostas percorridas: 16
Número de respostas percorridas: 17
Número de respostas percorridas: 18
Número de respostas percorridas: 19
Número de respostas percorridas: 20
Número de respostas percorridas: 21
Número de respostas percorridas: 22
Número de respostas percorridas: 23
Número de respostas percorridas: 24
Número de respostas percorridas: 25
Número de respostas percorridas: 26
Número de respostas percorridas: 27
Número de respostas percorridas: 28
N

In [19]:
# # Salvar embeddings em um arquivo .pkl (pickle)
# with open('final_responses.pkl', 'wb') as f:
#     pickle.dump(responses, f)

In [23]:
# Carregar embeddings do arquivo .pkl
with open('final_responses.pkl', 'rb') as f:
    responses = pickle.load(f)

print(f"Loaded Dataset size: {len(questions)}")

Loaded Dataset size: 150


In [24]:
responses[2]

'According to Information 1, Bill Dooley attended Perkinston Junior College in Perkinston, Mississippi from 1952 to 1953. He then moved on to Mississippi State University and graduated in 1956.\n\nTherefore, the state where Dooley went to college is:\nAnswer: Mississippi'

In [34]:
# Função para processar todo o dataset e calcular o F1 score
def evaluate_responses(responses, questions):
    total_f1 = 0
    f1_count_exact = 0
    f1_count_zero = 0
    num_responses = len(responses)

    for i in range(num_responses):
        
        answer = questions[i]['answer']
        answer = get_answer(answer)
        response = responses[i]
        
        # Calcular o F1 score
        f1 = compute_f1(answer, response)  # Passar somente a resposta
        total_f1 += f1
        
        if f1 == 1:
            f1_count_exact += 1
        elif f1 == 0:
            f1_count_zero += 1
        else:
            print(f"Resposta {i} - F1-Score: {f1}")
        
    
    average_f1 = total_f1 / num_responses if num_responses > 0 else 0
    return average_f1, f1_count_exact, f1_count_zero

In [47]:
# Avaliar usando respostas completas do Modelo

average_f1, exact_count, zero_count = evaluate_responses(responses, questions)

print("Média do F1 Score:", average_f1)
print("Número de vezes que o F1 Score foi igual a 1:", exact_count)
print("Número de vezes que o F1 Score foi igual a 0:", zero_count)

Resposta 1 - F1-Score: 0.021052631578947368
Resposta 38 - F1-Score: 0.6666666666666666
Resposta 39 - F1-Score: 0.017699115044247787
Resposta 49 - F1-Score: 0.024539877300613494
Resposta 53 - F1-Score: 0.5333333333333333
Resposta 54 - F1-Score: 0.0196078431372549
Resposta 56 - F1-Score: 0.33333333333333337
Resposta 59 - F1-Score: 0.19999999999999998
Resposta 67 - F1-Score: 0.002857142857142857
Resposta 69 - F1-Score: 0.045977011494252866
Resposta 73 - F1-Score: 0.01687763713080169
Resposta 82 - F1-Score: 0.025
Resposta 83 - F1-Score: 0.6666666666666666
Resposta 88 - F1-Score: 0.013245033112582783
Resposta 101 - F1-Score: 0.029411764705882356
Resposta 119 - F1-Score: 0.28571428571428575
Resposta 124 - F1-Score: 0.0003629105425512611
Resposta 143 - F1-Score: 0.16
Resposta 145 - F1-Score: 0.6666666666666666
Média do F1 Score: 0.20486007946190155
Número de vezes que o F1 Score foi igual a 1: 27
Número de vezes que o F1 Score foi igual a 0: 104


In [49]:
# Avaliar usando somente última linha da resposta do Modelo

average_f1, exact_count, zero_count = evaluate_responses(responses, questions)

print("Média do F1 Score:", average_f1)
print("Número de vezes que o F1 Score foi igual a 1:", exact_count)
print("Número de vezes que o F1 Score foi igual a 0:", zero_count)

Resposta 38 - F1-Score: 0.6666666666666666
Resposta 49 - F1-Score: 0.17391304347826084
Resposta 53 - F1-Score: 0.5333333333333333
Resposta 56 - F1-Score: 0.33333333333333337
Resposta 59 - F1-Score: 0.19999999999999998
Resposta 69 - F1-Score: 0.0625
Resposta 83 - F1-Score: 0.6666666666666666
Resposta 101 - F1-Score: 0.16666666666666669
Resposta 119 - F1-Score: 0.28571428571428575
Resposta 124 - F1-Score: 0.08333333333333333
Resposta 143 - F1-Score: 0.16
Resposta 145 - F1-Score: 0.6666666666666666
Média do F1 Score: 0.20665862663906143
Número de vezes que o F1 Score foi igual a 1: 27
Número de vezes que o F1 Score foi igual a 0: 111
